<p style="color: red; font-size: 16pt; font-weight: bold; text-align:center;">Change the name of this notebook before you edit!</p>

# Telecom Data

Source: https://www.kaggle.com/code/manishpuraswani/telecom-data-lr/input

In [1]:
! ls -lh /data/IFI8410/telecom/

total 1.2M
-rw-r--r--. 1 pmolnar pmolnar    480K Oct 23  2023 churn_data.csv
-rw-r--r--. 1 pmolnar pmolnar    185K Oct 23  2023 customer_data.csv
-rw-r--r--. 1 pmolnar pmolnar    456K Oct 23  2023 internet_data.csv
-rw-r--r--. 1 pmolnar united2024  162 Jan 30  2024 README.md


# Setup

In [2]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load data

In [3]:
customer_df = pd.read_csv('/data/IFI8410/telecom/customer_data.csv')
print(f"Number of customer records: {customer_df.shape[0]:,}")

Number of customer records: 7,042


In [4]:
internet_df = pd.read_csv('/data/IFI8410/telecom/internet_data.csv')
print(f"Number of internet records: {internet_df.shape[0]:,}")

Number of internet records: 7,042


In [5]:
churn_df = pd.read_csv('/data/IFI8410/telecom/churn_data.csv')
print(f"Number of internet records: {churn_df.shape[0]:,}")

Number of internet records: 7,042


## What does the data look like?

In [6]:
customer_df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents
0,7590-VHVEG,Female,0,Yes,No
1,5575-GNVDE,Male,0,No,No
2,3668-QPYBK,Male,0,No,No
3,7795-CFOCW,Male,0,No,No
4,9237-HQITU,Female,0,No,No


In [7]:
internet_df.head()

,customerID,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,7590-VHVEG,No phone service,DSL,No,Yes,No,No,No,No
1,5575-GNVDE,No,DSL,Yes,No,Yes,No,No,No
2,3668-QPYBK,No,DSL,Yes,Yes,No,No,No,No
3,7795-CFOCW,No phone service,DSL,Yes,No,Yes,Yes,No,No
4,9237-HQITU,No,Fiber optic,No,No,No,No,No,No


In [8]:
churn_df.head()

,customerID,tenure,PhoneService,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,1,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,34,Yes,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,2,Yes,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,45,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,2,Yes,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [9]:
customer_df['gender'].unique()

array(['Female', 'Male'], dtype=object)

AttributeError: 'DataFrame' object has no attribute 'unique'

## More Details

In [10]:
customer_df.dtypes

customerID       object
gender           object
SeniorCitizen     int64
Partner          object
Dependents       object
dtype: object

In [11]:
internet_df.dtypes

customerID          object
MultipleLines       object
InternetService     object
OnlineSecurity      object
OnlineBackup        object
DeviceProtection    object
TechSupport         object
StreamingTV         object
StreamingMovies     object
dtype: object

In [12]:
churn_df.dtypes

customerID           object
tenure                int64
PhoneService         object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

Data might need some cleaning:

`TotalCharges` is a string (object), should be a numerical value

In [13]:
jdf = pd.merge(internet_df, churn_df, on='customerID', how='inner') \
    .reset_index()
print(jdf.shape)

(7042, 18)


Let's compare features from two tables:

In [14]:
jdf.groupby(['InternetService', 'PaymentMethod'])['customerID'].apply('count')

InternetService  PaymentMethod            
DSL              Bank transfer (automatic)     566
                 Credit card (automatic)       594
                 Electronic check              648
                 Mailed check                  613
Fiber optic      Bank transfer (automatic)     645
                 Credit card (automatic)       597
                 Electronic check             1595
                 Mailed check                  258
No               Bank transfer (automatic)     332
                 Credit card (automatic)       331
                 Electronic check              122
                 Mailed check                  741
Name: customerID, dtype: int64

In [15]:
pd.pivot_table(jdf, 
               index='PaymentMethod', 
               columns='InternetService', 
               values='customerID',
               aggfunc='count')

InternetService,DSL,Fiber optic,No
PaymentMethod,,,
Bank transfer (automatic),566,645,332
Credit card (automatic),594,597,331
Electronic check,648,1595,122
Mailed check,613,258,741


## Deep Dive:
- Selecting rows and columns
- Joining tables vs extending/concatenating
    - inner, outer
    - merge() vs join()
- Adding new columns, copy sub-table
- Loading and saving data (why `index=None` ?)


In [19]:
df1 = pd.DataFrame(np.arange(12.).reshape((3, 4)), columns=list("abcd"))
df2 = pd.DataFrame(np.arange(20.).reshape((4, 5)), columns=list("abcde"))

In [20]:
df1.add(df2, fill_value=0)


,a,b,c,d,e
0,0.0,2.0,4.0,6.0,4.0
1,9.0,11.0,13.0,15.0,9.0
2,18.0,20.0,22.0,24.0,14.0
3,15.0,16.0,17.0,18.0,19.0


In [21]:
df2.add(df1, fill_value=0)

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,4.0
1,9.0,11.0,13.0,15.0,9.0
2,18.0,20.0,22.0,24.0,14.0
3,15.0,16.0,17.0,18.0,19.0


In [38]:
frame = pd.DataFrame(np.random.standard_normal((4, 3)), columns=list("bde"),
index=["Utah", "Ohio", "Texas", "Oregon"])

In [39]:
frame

,b,d,e
Utah,1.164957,-1.212807,0.330226
Ohio,1.228467,2.017266,1.201695
Texas,-0.456842,-1.262863,1.569273
Oregon,1.044706,-0.181038,0.850702


In [40]:
frame.describe()

,b,d,e
count,4.000000,4.000000,4.000000
mean,0.745322,-0.159860,0.987974
std,0.805057,1.534670,0.527592
min,-0.456842,-1.262863,0.330226
25%,0.669319,-1.225321,0.720583
50%,1.104831,-0.696923,1.026199
75%,1.180835,0.368538,1.293590
max,1.228467,2.017266,1.569273


In [36]:
frame['mean'] = frame.apply('mean', axis='columns')
frame

,b,d,e,max_min,mean
Utah,1.028181,0.013580,-0.708831,1.737012,0.517485
Ohio,1.099720,-0.935930,0.589496,2.035650,0.697234
Texas,-0.559983,0.804020,-0.317882,1.364003,0.322540
Oregon,0.582020,0.896233,0.339098,0.557135,0.593621


In [37]:
frame['b'].mean()

0.5374843246942936

In [41]:
frame['b'].corr(frame['d'])

0.520617323729918

In [43]:
frame['b'].corr(frame['e'])

-0.7162536250497118

In [42]:
frame['b'].cov(frame['d'])

0.6432215905987547

In [24]:
def f1(x):
    return x.max() - x.min()

In [25]:
frame.apply(f1)

b    1.659703
d    1.832163
e    1.298327
dtype: float64

In [28]:
frame2 = frame.copy()

In [29]:
frame2['max_min'] = frame2.apply(f1, axis="columns")

In [30]:
frame2

,b,d,e,max_min
Utah,1.028181,0.013580,-0.708831,1.737012
Ohio,1.099720,-0.935930,0.589496,2.035650
Texas,-0.559983,0.804020,-0.317882,1.364003
Oregon,0.582020,0.896233,0.339098,0.557135


In [31]:
frame['max_min'] = frame.apply(f1, axis="columns")

In [32]:
frame

,b,d,e,max_min
Utah,1.028181,0.013580,-0.708831,1.737012
Ohio,1.099720,-0.935930,0.589496,2.035650
Texas,-0.559983,0.804020,-0.317882,1.364003
Oregon,0.582020,0.896233,0.339098,0.557135
